## Homework 7.2
Heredity in Darwin’s finches, the parametric way

In [ ]:
# Colab setup ------------------
import os, sys, subprocess
if "google.colab" in sys.modules:
    cmd = "pip install --upgrade iqplot colorcet watermark"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    data_path = "https://s3.amazonaws.com/bebi103.caltech.edu/data/"
else:
    data_path = "../data/"
# ------------------------------
import warnings
import numpy as np
import pandas as pd
import scipy.stats as st
import scipy.optimize

import numba
import tqdm
import bebi103

import iqplot

import colorcet

import bokeh.io
bokeh.io.output_notebook()

First, we load in the beak data and format it for our convenience by creating a column with the average adult Fortis beak depth and renaming the column for the average offspring beak depth to make it consistent with the scandens data.  

In [ ]:
fname = os.path.join(data_path, "fortis_beak_depth_heredity.csv")
fortis_df = pd.read_csv(fname, comment="#")
fortis_df["mid_parent"] = (fortis_df["Male BD"] + fortis_df["Female BD"]) / 2
fortis_df = fortis_df.rename(columns={"Mid-offspr": "mid_offspring"})
fortis_df.head()

We also load in the Scandens species data. 

In [ ]:
fname = os.path.join(data_path, "scandens_beak_depth_heredity.csv")
scandens_df = pd.read_csv(fname, comment="#")
scandens_df.head()

In this problem, we aim to compute a maximum likelihood estimate for heritability using a parametric approach by modeling parental and offspring beak depths using a bivariate normal distribution. We define a 2-vector containing the parental and offspring beak depths. Our model is

\begin{align}
\mathbf{y}_i \sim \text{Norm}(\boldsymbol{\mu}, \mathsf{\Sigma})\;\;\;\forall i,
\end{align}

where $\boldsymbol{\mu} = (\mu_p, \mu_o)^\mathsf{T}$ is a vector containing the mean parent and offspring beak depth and $\mathsf{\Sigma}$ is the covariance matrix,

\begin{align}
\mathsf{\Sigma} = \begin{pmatrix}
\sigma_p^2 & \sigma_{op} \\
\sigma_{op} & \sigma_o^2
\end{pmatrix}.
\end{align}

Therefore, in our model there are 5 parameters:  $\mu_p$, $\mu_o$, $\sigma_p$, $\sigma_o$, and $\sigma_{op}$.

We first begin by defining `log_like_iid_mnorm`, a function that calculates the log likelihood for the multivariate normal distribution using the scipy stats module. We use  $\mu_p$, $\mu_o$, $\sigma_p$, $\sigma_o$, and $\sigma_{op}$ as our parameters but reassign them to `mu` and `sigma` in an array and matrix respectively for the scipy module. For all code from here, we say that $\mu_p$, $\mu_o$ are represented by variables mu1 and mu2, while $\sigma_p$, $\sigma_o$, and $\sigma_{op}$ are represented by sig1, sig2, and sig3 respectively. Note that the MLE for the covariance matrix and the mean vector in a bivariate normal can be calculated in the same fashion as those statistics (see the following link for reference: https://people.eecs.berkeley.edu/~jordan/courses/260-spring10/other-readings/chapter13.pdf).

For disallowed parameter values, such as when mu or sigma <= 0, we return -np.inf because this is the limit of a logarithm as the value approaches 0 from the right side - this is consistent with the Powell method. We also test that the covariance matrix is a positive definite matrix, and if not similarly return -np.inf.


In [ ]:
#Code based on Bois (2020)

def log_like_iid_mnorm(params, n):
    """Log likelihood for i.i.d. Multivariate Norm measurements, parametrized
    by mu, Sigma (covariance matrix)."""
    mu1, mu2, sig1, sig2, sig3 = params
    mu = [mu1, mu2]
    sig = np.asmatrix([[sig1, sig3], [sig3, sig2]])
    if mu[0] <= 0 or mu[1] <= 0: 
        return -np.inf
    if sig1 <= 0 or sig2 <= 0: 
        return -np.inf
    # test if covariance matrix is positive definite
    try: 
        np.linalg.cholesky(sig)
    except:
        return -np.inf
    return np.sum(st.multivariate_normal.logpdf(n, np.asarray(mu), sig))

We then make a function that calculates maximum likelihood estimates for parameters for i.i.d Multivariate normal measurements, parametrized by mu, sigma (cov matrix).

In [ ]:
#Code based on Bois (2020)

def mle_iid_mnorm(n):
    """Perform maximum likelihood estimates for parameters for i.i.d.
    Multivariate Normal measurements, parametrized by mu, sigma (cov matrix)"""
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")

        res = scipy.optimize.minimize(
            fun=lambda params, n: -log_like_iid_mnorm(params, n),
            x0=np.array([8, 8, 1, 2, 0.5]),
            args=(n,),
            method='Powell'
        )

    if res.success:
        return res.x
    else:
        raise RuntimeError('Convergence failed with message', res.message)

In order to perform MLE for each species, we first create a tidy dataframe, that contains the averages of beak depth for the parents and respective offspring for each species. 

In [ ]:
species = ["fortis", "scandens"] 
df1 = pd.DataFrame(fortis_df[['mid_parent', 'mid_offspring']])
df1['species'] = "fortis"
df2 = pd.DataFrame(scandens_df[['mid_parent', 'mid_offspring']])
df2['species'] = "scandens"
master = pd.concat([df1, df2])
master

Before performing MLE, we create an empty dataframe, `df_mle`, that contains columns for the species, the parameter being estimated, and the mle value. We will fill this dataframe with out MLE estimates; it is empty currently. 

In [ ]:
df_mle = pd.DataFrame(columns=['species', 'parameter', 'mle'])
df_mle

To perform MLE, we use our function defined earlier, `mle_iid_mnorm` for each of the species. We use a for loop that creates a sub dataframe that stores the species, the parameter value being estimated, and the actual mle value. We append this information in `df_mle` as its computed. 

In [ ]:
# Perform MLE for each species
for sp in species:
    mle = mle_iid_mnorm(master.loc[master['species'] == sp].values[:,:2])
    sub_df = pd.DataFrame({'species': [sp]*5, 'parameter': ['mu1', 'mu2', 'sig1', 'sig2', 'sig3'], 'mle': mle})
    df_mle = df_mle.append(sub_df)
df_mle

Finally, to calculate the confidence intervals for these parameter estimates we use parametric bootstrapping. 

We use a function made by Bois(2020) that draws parametric replicates of MLE. We also need to a provide a function that draws new data sets our of the parametric model, and thus make a function `sp_multinorm` that return a multivariate normal distribution given the 5 parameters  $\mu_p$, $\mu_o$, $\sigma_p$, $\sigma_o$, and $\sigma_{op}$ and size, using the Numpy random module.

In [ ]:
#Set up Numpy random generator
rg = np.random.default_rng()

def draw_parametric_bs_reps_mle(
    mle_fun, gen_fun, data, args=(), size=1, progress_bar=False
):
    """Draw parametric bootstrap replicates of maximum likelihood estimator.

    Parameters
    ----------
    mle_fun : function
        Function with call signature mle_fun(data, *args) that computes
        a MLE for the parameters
    gen_fun : function
        Function to randomly draw a new data set out of the model
        distribution parametrized by the MLE. Must have call
        signature `gen_fun(*params, size)`.
    data : one-dimemsional Numpy array
        Array of measurements
    args : tuple, default ()
        Arguments to be passed to `mle_fun()`.
    size : int, default 1
        Number of bootstrap replicates to draw.
    progress_bar : bool, default False
        Whether or not to display progress bar.

    Returns
    -------
    output : numpy array
        Bootstrap replicates of MLEs.
    """
    params = mle_fun(data, *args)

    if progress_bar:
        iterator = tqdm.tqdm(range(size))
    else:
        iterator = range(size)

    return np.array(
        [mle_fun(gen_fun(*params, size=len(data), *args)) for _ in iterator]
    )

#Generates samples from the model distribution.
def sp_multinorm(mu1, mu2, sig1, sig2, sig3, size):
    mu = [mu1, mu2]
    sig = np.asmatrix([[sig1, sig3], [sig3, sig2]])
    return rg.multivariate_normal(mu, sig, size=size)

Here we take the parametric bootstraps for the Fortis species. The output is an array of the confidence intervals. Thus the lower bound of the confidence interval for the parameters while the second array is the upper bound of the confidence interval. 

In [ ]:
bs_reps_parametric = draw_parametric_bs_reps_mle(
    mle_iid_mnorm,
    sp_multinorm,
    master.loc[master['species'] == "fortis"].values[:,:2],
    args=(),
    size=10000,
    progress_bar=True,
)
np.percentile(bs_reps_parametric, [2.5, 97.5], axis=0)

Our results here are consistent with the results in the previous problem because our parameter estimates in the previous lie within the confidence intervals calculated here.  To visualize the relationship between the parameters we can plot the bootstrap samples from each of the parameters against the others. 

In [ ]:
# Package replicates in data frame for plotting
df_res = pd.DataFrame(data=bs_reps_parametric, columns=["mu1", "mu2", "sig1", "sig2", "sig3"])

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    p = bebi103.viz.corner(
        samples=df_res,
        parameters=["mu1", "mu2", "sig1", "sig2", "sig3"],
        show_contours=True,
        levels = [0.95],
    )

bokeh.io.show(p)

In these figures, each parameter is compared against the other parameters. We can see that that the means between offspring and parents are positively correlated because we can see a linear relationship. This makes sense because as beak depth increases in parents, given a certain level of heritability, beak depth of offsprings should also increase. We can also see that it seems like $\sigma_p$, $\sigma_o$, are positively correlated with $\sigma_{op}$. This could be the case because as an increase in variance in the offspring and parents would lead to an increased covariance between offspring and parents. Since beak depth is quite heritable in the fortis species, having an increased variance in parents would lead to increased variance in the offspring, and therefore increased covariance. 

Next, we take the parametric bootstraps to calculate the confidence intervals for the MLE estimates in the scandens species. 

In [ ]:
bs_reps_parametric1 = draw_parametric_bs_reps_mle(
    mle_iid_mnorm,
    sp_multinorm,
    master.loc[master['species'] == "scandens"].values[:,:2],
    args=(),
    size=1000,
    progress_bar=True,
)
np.percentile(bs_reps_parametric1, [2.5, 97.5], axis=0)

Again, our are consistent with the results in the previous problem because our parameter estimates in the previous lie within the confidence intervals calculated here.  To visualize the relationship between the parameters we can plot the bootstrap samples from each of the parameters against the others. 

In [ ]:
# Package replicates in data frame for plotting
df_res1 = pd.DataFrame(data=bs_reps_parametric1, columns=["mu1", "mu2", "sig1", "sig2", "sig3"])

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    p1 = bebi103.viz.corner(
        samples=df_res1,
        parameters=["mu1", "mu2", "sig1", "sig2", "sig3"],
        show_contours=True,
        levels = [0.95],
    )

bokeh.io.show(p1)

Compared to the plots for fortis, the plots show that the parameters of scandens are less correlated to one another - we would expect that parameters that are correlated to show linear relationships but we see dispersed distributions suggesting less correlation between the parameters. Even comparing the means of offpspring vs parents, we don't see a distinct linear correlation as we could see in the fortis plot. This is consistent with the idea that fortis beak depth is more heritible than scandens beak depth. 


In [ ]:
'''
    Attribution: Jen wrote most of the code with input from Arun and Sammy. 
                 All three of us worked on the analysis and solution discussion.
'''

%load_ext watermark
%watermark -v -m -p jupyterlab,numpy,pandas,scipy,bokeh,iqplot,numba,tqdm,colorcet

<div class="alert alert-info">
    
Good work computing the plug-in and MLE estimates! However, the problem asked for confidence intervals for heritability, and it looks like you only computed and showed confidence intervals for the 5 parameters. You would have just had do one more step and compute heritability from the variances of parents and offspring. 
    
-2
    
Grade: 28/30
    
</div>